<a href="https://colab.research.google.com/github/MariaVlasenkova/repo-PNM/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D0%B2%D1%82%D0%BE%D1%80%D0%BE%D0%B3%D0%BE_%D1%8D%D1%82%D0%B0%D0%BF%D0%B0_(%D0%BC%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача II.1.3.1. (40 баллов)

# Условие
На Олимпиаду НТИ приехали участники, некоторые из которых знакомы, часть которых знакома между собой, а другая – нет.
Участников необходимо разбить на команды, минимизировав количество команд.
В команде может быть от одного до M участников включительно.
При этом каждый член команды должен знать хотя бы одного другого члена команды (если в команде больше одного человека).
Считается, что участники пронумерованы начиная с нуля. На вход подается пара:
1.	Cписок из N массивов по N значений. i-e значение в j-м массиве, означает знакомы ли участники под номерами i и j: 1 – если знакомы и 0 – иначе. Легко заметить, что матрица симметрична, относительно главной диагонали.
2.	Максимальное количество участников в команде M. 
N изменяется от 50 до 70 (в примере равно 10). M изменяется от 3 до 7.
На выходе необходимо написать одно число – минимальное количество команд при заданном ограничении.

Количество попыток неограниченно, ограничение по времени работы зависит от языка.
Немного занимательной информации о знакомстве участников: всегда существует такой порядок, что выстроившись в нем в линию каждый участник, который имеет хотя бы одного знакомого сможет пожать руку какому-то знакомому, т.е. будет иметь знакомого или справа или слева от себя. Существование такого порядка можно использовать в алгоритме.

# Генерация данных

In [8]:
#cлучайная генерация данных, с указанными ограничнениями
def generate():
  examples = 10
  resl = list()
  for ex in range(examples):
    maxa = random.randint(50,70)
    maxb=maxa
    clust_num = random.randint(int(maxa/8),int(maxa/5 ))
    clust_sizes = [random.randint(int(maxa/clust_num-5),int(maxa/clust_num+5)) for i in range(clust_num-2)]
    if sum(clust_sizes) > maxa-3:
      clust_sizes = [random.randint(int(maxa/clust_num-5),int(maxa/clust_num)) for i in range(clust_num-3)]
    sumcl = sum(clust_sizes)
    finr = random.randint(0,maxa-sumcl)
    clust_sizes.append(finr)
    clust_sizes.append(maxa - sumcl-finr)
    chikld = [i for i in range(maxa)]
    cls_ids = []
    res = [[0]*maxa for i in range(maxb)]
    for cl in clust_sizes:
      if cl<1:
        continue
      cl_ids = []
      for i in range(cl):
        rind = random.randint(0,len(chikld)-1)
        cl_ids.append(chikld[rind])
        chikld.remove(chikld[rind])
      for i in range(1,len(cl_ids)):
        res[cl_ids[i]][cl_ids[i-1]] = 1
        res[cl_ids[i-1]][cl_ids[i]] = 1
      for i in range(int(len(cl_ids))*5):
        if len(cl_ids)<1:
          continue
        i1 = cl_ids[random.randint(0,len(cl_ids)-1)]
        i2 = cl_ids[random.randint(0,len(cl_ids)-1)]
        if i1==i2:
          continue
        con = max( res[i1][i2], random.randint(0,1))
        res[i1][i2] = con
        res[i2][i1] = con
      cls_ids.append(list(set(cl_ids)))
    res = [ [1 if j==i
        else res[i][j] for i in range(maxa)] for j in range(maxb)]
    cls_ids = sorted(cls_ids, key=lambda x: x[0])
    resl.append( "("+str(res)+","+str(random.randint(3,7)) +")"+"\n")
  return resl

# Решение

In [7]:
import random
import math
import ast

#получить кластер от конкретного участника
def getcluster_from_point(r,p,cluster=[]):
  cluster = list(set(cluster+[p]))
  new_ps = [i for i in range(len(r[0])) if r[p][i]==1 and i not in cluster]
  cluster = list(set(cluster+new_ps))
  for new_p in new_ps:
    clustern = getcluster_from_point(r,new_p, cluster=cluster)
    cluster = list(set(clustern))
  return cluster

#получить все кластера
def get_clusters(r):
  points = [i for i in range(len(r[0]))]
  clusters = []
  while len(points)>0:
    clustern = getcluster_from_point(r,points[0])
    #print(clustern,points[0])
    points = [p for p in points if p not in clustern]
    #print(points,  clustern,  len(points))
    clusters.append(clustern)
  clusters = sorted(clusters, key=lambda x: x[0])
  return clusters

#минимизация количества команд
def get_min(m, r):
  cls = get_clusters(r)
  cum = 0
  for cl in cls:
    cum+=int(math.trunc(len(cl)/m))
    if len(cl) %m >0:
      cum+=1
  return str(cum)

#обработка входа и выдача результата
def solve(dataset):
  r,m = ast.literal_eval(dataset)
  return get_min(m, r)